In [2]:
! pip install -r requirements.txt --quiet

# Creating and Deploying an Azure AI Agent Using SDK and OpenAPI Tools

This notebook demonstrates how to create and deploy an Azure AI Agent using the Azure SDK, with tools generated from an OpenAPI endpoint. It provides a coded example that mirrors the portal example demonstrated in this workshop, offering a programmatic approach to deploying agents with OpenAPI-driven tools.


🔗 [How to use Azure AI Agent Service with OpenAPI Specified Tools](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/openapi-spec?tabs=python&pivots=code-example)

In [1]:

from os import environ
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import OpenApiTool, OpenApiAnonymousAuthDetails
import requests

load_dotenv(override=True)

True

## Create a project client

In [2]:

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Create the OpenAPI Spec tool definition

Our Agent will create its tools from our deployed OpenAPI endpoints, for this to work we will need to pass in the openapi.json spec document.

In [3]:
spec_url = environ['OPENAPI_URL']
response = requests.get(spec_url)
response.raise_for_status()


openapi_spec = response.json()

auth = OpenApiAnonymousAuthDetails()

openapi = OpenApiTool(name="AutomotiveSalesService", spec=openapi_spec, description="Retrieve automotive sales data for query and advanced analytics", auth=auth)




## Create an agent and a thread

In [4]:
instructions = """You are AutoSales Analyst, an AI agent specialized in analyzing automotive sales data.

You have access to the following OpenAPI endpoints as tools:
1. /orders: Retrieve sales orders with nested order lines and product details.
   - Supports filters: customer_id, product_id, start_date, end_date, region.
2. /products: Retrieve product catalog, optionally filtered by category.
3. /customers: Retrieve customer data.

Your objectives:
- Interpret user questions about sales data and dynamically decide which endpoints to call.
- Always include the customer name along with the customer ID in any output.
- Always show all monetary amounts in USD (e.g., $123.45).
- Use filters and aggregations on /orders, /products, and /customers to generate insights.
- Compute totals, revenue, discounts, and other metrics from the nested order data.
- Return structured, concise results suitable for analysis or reporting.
- Avoid hardcoding analytics; instead, dynamically use the endpoints and their filters.
- Clarify ambiguous user queries before performing analysis.
- Always assume the endpoints are the source of truth; do not expose raw database internals.

Examples of user questions you may handle:
- “Which products generated the most revenue in the EU last quarter?”
- “Who are the top 5 customers by total sales this month?”
- “How much discount was given per product category in Q2?”
- “List all orders for customer 42 containing product 7.”
"""



agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="Sales Insights Agent OpenAPI",
    instructions=instructions,
    tools=openapi.definitions
)

print(f"Created agent, ID: {agent.id}")

# Create thread for communication
#thread = project_client.agents.threads.create()
#print(f"Created thread, ID: {thread.id}")



Created agent, ID: asst_og9q1JWLHgXbgZxj6TicGE0a


## Create a run and check the output

In [5]:
#message = project_client.agents.messages.create(
#    thread_id=thread.id,
#    role="user",
#    content="Which 3 customers have the highest sales for the year 2024?",
#)
#print(f"Created message, ID: {message.id}")

# Create and process agent run in thread with tools
#run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
#print(f"Run finished with status: {run.status}")

#if run.status == "failed":
#    print(f"Run failed: {run.last_error}")

# Fetch and log all messages
#messages = project_client.agents.messages.list(thread_id=thread.id)
#print(f"Messages: {messages}")

## Delete the assistant when done

Comment the code below to save the agent to your Azure AI Froundry project.


In [6]:

#project_client.agents.delete_agent(agent.id)
#print("Deleted agent")